In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import scipy

import clmm
from clmm import Cosmology
from clmm import GalaxyCluster
from clmm.dataops import compute_galaxy_weights
from clmm.support import mock_data as mock
clmm.__version__

In [ ]:
cosmo = Cosmology(H0 = 71.0, Omega_dm0 = 0.265 - 0.0448, Omega_b0 = 0.0448, Omega_k0 = 0.0)

In [ ]:
cluster_z = 0.4

In [ ]:
noisy_data_z = mock.generate_galaxy_catalog(1e14, cluster_z, 4, 
                                                cosmo, 
                                                zsrc = 'chang13', 
                                                #Delta_SO=200, 
                                                massdef='critical',
                                                halo_profile_model='nfw', 
                                                zsrc_min=0.,
                                                zsrc_max=3., 
                                                field_size=10., 
                                                shapenoise=0.5, 
                                                photoz_sigma_unscaled=0.05, 
                                                ngals=10000)
z_gal = noisy_data_z['ztrue']
z_gal_photoz = noisy_data_z['pzbins']
photoz_pdf_gal = noisy_data_z['pzpdf']
e1 = noisy_data_z['e1']
e2 = noisy_data_z['e2']
noisy_data_z['e1_err'] = np.random.randn(len(e1))*0.02 + 0.1*(1 + z_gal)
noisy_data_z['e2_err'] = np.random.randn(len(e1))*0.02 + 0.03*(1 + z_gal)
cl = clmm.GalaxyCluster('mock_cluster', 0, 0, .4, noisy_data_z)

### WL weights (photoz + shape errors)

as a function

In [ ]:
w_ls_true = compute_galaxy_weights(
    cluster_z, cosmo,z_source = z_gal, 
    shape_component1 = e1, shape_component2 = e2, 
    shape_component1_err = noisy_data_z['e1_err'],
    shape_component2_err = noisy_data_z['e2_err'], 
    pzpdf = None, pzbins = None,
    add_shapenoise = False, is_deltasigma = False)

as a `class` function of the GalaxyCluster object

In [ ]:
w_ls_true = cl.compute_galaxy_weights(
    z_source='ztrue', pzpdf='pzpdf', pzbins='pzbins', 
    shape_component1 = 'e1', shape_component2 = 'e2', 
    shape_component1_err = 'e1_err', shape_component2_err = 'e2_err', 
    add_photoz = False, add_shapenoise = False, add_shape_error = False, 
    weight_name = 'w_ls_true', p_background_name = 'p_background_true',
    cosmo = cosmo, is_deltasigma = True, add = True)

In [ ]:
w_ls_photoz = cl.compute_galaxy_weights(
    z_source='ztrue', pzpdf='pzpdf', pzbins='pzbins', 
    shape_component1 = 'e1', shape_component2 = 'e2', 
    shape_component1_err = 'e1_err', shape_component2_err = 'e2_err', 
    add_photoz = True, add_shapenoise = False, add_shape_error = False, 
    weight_name = 'w_ls_photoz', p_background_name = 'p_background_photoz',
    cosmo = cosmo, is_deltasigma = True, add = True)

In [ ]:
w_ls_shape = cl.compute_galaxy_weights(
    z_source='ztrue', pzpdf='pzpdf', pzbins='pzbins', 
    shape_component1 = 'e1', shape_component2 = 'e2', 
    shape_component1_err = 'e1_err', shape_component2_err = 'e2_err', 
    add_photoz = False, add_shapenoise = True, add_shape_error = True, 
    weight_name = 'w_ls_shape', p_background_name = 'p_background_shape',
    cosmo = cosmo,
    is_deltasigma = True, add = True)

In [ ]:
w_ls_photoz_shape = cl.compute_galaxy_weights(
    z_source='ztrue', pzpdf='pzpdf', pzbins='pzbins', 
    shape_component1 = 'e1', shape_component2 = 'e2', 
    shape_component1_err = 'e1_err', shape_component2_err = 'e2_err', 
    add_photoz = True, add_shapenoise = True, add_shape_error = True, 
    weight_name = 'w_ls_photoz_shape', p_background_name = 'p_background_photoz_shape',
    cosmo = cosmo,
    is_deltasigma = True, add = True)

In [ ]:
plt.rcParams['axes.linewidth'] = 1
fig, ax = plt.subplots(1,2, figsize = (10,5), sharey = True)
fig.subplots_adjust(wspace=0, hspace=0)
ax[0].tick_params(axis='both', which = 'major', labelsize= 10)
ax[1].tick_params(axis='both', which = 'major', labelsize= 10)
ax[0].scatter(z_gal, 1e3*cl.galcat['w_ls_true']/np.sum(cl.galcat['w_ls_true']), c='r', label = 'True z', zorder = 100)
ax[0].scatter(z_gal, 1e3*cl.galcat['w_ls_photoz']/np.sum(cl.galcat['w_ls_photoz']), c='orange', label = 'Photoz')
ax[1].scatter(z_gal, 1e3*cl.galcat['w_ls_shape']/np.sum(cl.galcat['w_ls_shape']), c='royalblue', label = r'Shape_error', zorder = 100)
ax[1].scatter(z_gal, 1e3*cl.galcat['w_ls_photoz_shape']/np.sum(cl.galcat['w_ls_photoz_shape']), c='cyan', label ='Shape_error + Photoz')
ax[0].legend(frameon = False, fontsize = 13)
ax[1].legend(frameon = False, fontsize = 13)
ax[0].set_ylabel(r'p $(\times 10^{-3})$', fontsize = 20)
ax[0].set_xlabel('true redshift', fontsize = 14)
ax[1].set_xlabel('true redshift', fontsize = 14)
#plt.savefig('/pbs/throng/lsst/users/cpayerne/ThesisAtCCin2p3/Images/sprint_week.png', bbox_inches='tight', dpi=300)

### Background probability

In [ ]:
noisy_data_z_2 = mock.generate_galaxy_catalog(1e14, cluster_z, 4, 
                                                cosmo, 
                                                zsrc = 'chang13', 
                                                #Delta_SO=200, 
                                                massdef='critical',
                                                halo_profile_model='nfw', 
                                                zsrc_min=0.,
                                                zsrc_max=3., 
                                                field_size=10., 
                                                shapenoise=0.5, 
                                                photoz_sigma_unscaled=0.03, 
                                                ngals=10000)
z_gal_2 = noisy_data_z_2['ztrue']
z_gal_photoz_2 = noisy_data_z_2['pzbins']
photoz_pdf_gal_2 = noisy_data_z_2['pzpdf']
e1_2 = noisy_data_z_2['e1']
e2_2 = noisy_data_z_2['e2']
noisy_data_z_2['e1_err'] = np.random.randn(len(e1_2))*0.02 + 0.1*(1 + z_gal_2)
noisy_data_z_2['e2_err'] = np.random.randn(len(e1_2))*0.02 + 0.03*(1 + z_gal_2)
cl_2 = clmm.GalaxyCluster('mock_cluster', 0, 0, cluster_z, noisy_data_z_2)

In [ ]:
w_ls_photoz = cl_2.compute_galaxy_weights(z_source='ztrue', pzpdf='pzpdf', pzbins='pzbins', 
                        shape_component1 = 'e1', shape_component2 = 'e2', 
                        shape_component1_err = 'e1_err', shape_component2_err = 'e2_err', 
                        add_photoz = True, add_shapenoise = False, add_shape_error = False, 
                        weight_name = 'w_ls_photoz', p_background_name = 'p_background_photoz',
                        cosmo = cosmo,
                        is_deltasigma = True, add = True)

In [ ]:
plt.rcParams['axes.linewidth'] = 1
plt.figure( figsize = (7,5))
plt.tick_params(axis='both', which = 'major', labelsize= 10)
plt.plot([],[],' ', label = f'$z_l = {cluster_z}$', )
plt.scatter(z_gal,cl .galcat['p_background_photoz'], c='royalblue', label = r'$\sigma_z = 0.05$')
plt.scatter(z_gal_2, cl_2.galcat['p_background_photoz'], c='cyan', label = r'$\sigma_z = 0.03$')
plt.vlines(cluster_z, 0,1,color='salmon', linewidth = 4, zorder = 100)
plt.scatter(z_gal, p_background_true, c='salmon', s = 3, zorder = 110)
plt.plot([],[], label = 'True z', linewidth = 4, c = 'salmon')
plt.xlim(0., 1)
plt.legend(frameon = True, fontsize = 20)
plt.ylabel(r'$P(z_s > z_l)$', fontsize = 20)
plt.xlabel('true redshift', fontsize = 20)
#plt.savefig('/pbs/throng/lsst/users/cpayerne/ThesisAtCCin2p3/Images/sprint_week_p_background.png', bbox_inches='tight', dpi=300)